In [24]:
import numpy as np
import pandas as pd
import math
import torch
import os
import matplotlib.pyplot as plt
from scipy.signal import hilbert,chirp
from torch import nn
from d2l import torch as d2l
%matplotlib inline

In [41]:
os.chdir(r'C:\\Users\\86136\\PycharmProjects\\pythonProject\\AI -learn from zero\\data\\sensor_data_origin\\0928_cycle10s_3v_0.5v')
##  要切换到的新路径  可以%pwd 查看当前路径
file_chdir = os.getcwd() ##  获取当前路径
file_name_list=[]
file_list=[] 
for root,dirs,files in os.walk(file_chdir):  ## file_chdir :代表需要遍历的根文件夹  root :表示正在遍历的文件夹的名字（根/子）
                                            ## dirs :记录正在遍历的文件夹下的子文件夹集合  files:记录正在遍历的文件夹中的文件集合(list形式)
    for file in files:
        if os.path.splitext(file)[-1] =='.csv': ## os.path.splitext()  分离文件名与扩展名；默认返回(fname,fextension)元组 切片后-1 表示后缀
            file_name_list.append(file)
            file_list.append(pd.read_csv(file))


In [48]:
## 添加位命名的列
for num,file in enumerate(file_list):
      file.rename(columns={'Unnamed: 8': 'humidity'},inplace=True)
    

In [52]:
## 可以设置接口 手动输入 周期的数量和 目标训练数据数量
cycle_data_length = 1000 # 采样率为 100Hz 周期为10s
traget_data_length = 70
count =[]
## 计算出不同表格所包含的脉冲数量
for num,file in enumerate(file_list):
    count.append(math.floor(len(file_list[num])/cycle_data_length))

In [82]:
## 创建 2维矩阵  第一维代表 表格个数  第二维代表脉冲定位的点数 （这里是初始化）  长度代表脉冲宽度
z=[]
for i in range(len(count)):
    z.append([0]*count[i])

In [99]:
## 寻找阶跃的点 并且将位置信息存在  z 列表中
for num ,counts in enumerate(count):
    for j in range(counts):
        for i in range(j*cycle_data_length,(j+1)*cycle_data_length):
            if file_list[num].loc[i," pulse "] > 2.5 and file_list[num].loc[(i-1)," pulse "] < 0.5:
                z[num][j]=file_list[num].index[i]        
       

In [105]:
## 创建目标的dataFrame 用于输出
df = []
for i in range(len(count)):
    df.append(pd.DataFrame(np.zeros(file_list[i].shape), index=file_list[i].index, columns=file_list[i].columns))

pandas.core.frame.DataFrame

In [107]:
## 将脉冲数据 按格式输出 
for num,down_l in enumerate(z):
    for lb,down in enumerate(down_l):
        for j in range(traget_data_length):
            df[num].iloc[lb*traget_data_length+j,:] = file_list[num].iloc[down-20+j,:]
    
#     print(z[num].index(down_l))
#     for j in range(traget_data_length):
#         df[num].loc[down_l*traget_data_length+j,:] = file_list[num].iloc[num-20+j,:]


In [113]:
## 去掉全为0的行
for i in range(len(count)):
    df[i]=df[i].loc[(df[i]!=0).any(1)]

In [118]:
##将文件按格式保存
typing = input("输入本次处理的气体类型（英文）：")
for i in range(len(count)):
    df[i].to_csv(f'dataset{i}0ppm_{typing}.csv')

输入本次处理的气体类型（英文）：ethanol


In [199]:
for i in range(len(count)):
    
    for j in range (1,5):
        exec('ppm{}0sensor{}= df[{}].iloc[:,{}].values.reshape(-1,traget_data_length)'.format(i,j,i,j) ) ## serial 没有 reshape 功能
        exec('ppm{}0sensor{}= pd.DataFrame(ppm{}0sensor{})'.format(i,j,i,j))
        exec('ppm{}0sensor{}.loc[:,traget_data_length+1] = {}0'.format(i,j,i))
        exec('ppm{}0sensor{}.rename(columns={},inplace=True)'.format(i,j,{70:"humidity(v)"}))
        exec('ppm{}0sensor{}.rename(columns={},inplace=True)'.format(i,j,{71:"label(ppm)"}))
        #exec('print(len(ppm{}0sensor{}))'.format(i,j))
        exec('ppm{}0sensor{}=ppm{}0sensor{}.iloc[math.ceil(len(ppm{}0sensor{})/6):math.ceil(5*len(ppm{}0sensor{})/6),:]'.format(i,j,i,j,i,j,i,j))
        exec('ppm{}0sensor{}.to_csv(\'{}0ppms{}_dset.csv\')'.format(i,j,i,j))

In [200]:
ppm10sensor4

,0,1,2,3,4,5,6,7,8,9,...,61,62,63,64,65,66,67,68,69,label(ppm)
60,0.255806,0.255806,0.255162,0.256128,0.255806,0.255484,0.254195,0.256451,0.254517,0.257095,...,0.257095,0.255806,0.255806,0.254517,0.254517,0.253551,0.253551,0.253551,0.252584,10
61,0.253873,0.254517,0.254840,0.255162,0.254517,0.254517,0.254840,0.255162,0.254840,0.254195,...,0.255162,0.254840,0.253551,0.253551,0.253551,0.253551,0.252906,0.251940,0.251940,10
62,0.253551,0.254195,0.253551,0.253873,0.253551,0.253229,0.253873,0.252906,0.253873,0.253551,...,0.253873,0.252584,0.252262,0.251940,0.252262,0.251940,0.251617,0.252262,0.251617,10
63,0.252584,0.252906,0.252262,0.252584,0.252262,0.251940,0.252906,0.252584,0.253229,0.252584,...,0.252906,0.252262,0.251295,0.251295,0.250973,0.251295,0.250329,0.250651,0.250329,10
64,0.251617,0.251940,0.250973,0.250973,0.251617,0.251295,0.251617,0.250973,0.251617,0.251295,...,0.252906,0.251295,0.250973,0.251295,0.250651,0.250006,0.250651,0.250329,0.249040,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,0.203930,0.203930,0.203930,0.204252,0.203607,0.204252,0.203607,0.205541,0.203607,0.203607,...,0.205218,0.204574,0.203607,0.204252,0.204896,0.201030,0.204252,0.202319,0.203607,10
296,0.204896,0.203607,0.204574,0.203607,0.204252,0.203607,0.203607,0.204252,0.203285,0.203930,...,0.205218,0.204896,0.203607,0.203930,0.203607,0.203285,0.203607,0.201996,0.202641,10
297,0.203285,0.203607,0.204574,0.203607,0.203607,0.203930,0.204252,0.203607,0.203930,0.203607,...,0.204896,0.203607,0.203285,0.202319,0.202963,0.202319,0.202641,0.202641,0.201996,10
298,0.203285,0.202641,0.203285,0.203285,0.203607,0.203607,0.204252,0.202963,0.203930,0.203285,...,0.204574,0.203930,0.203607,0.203930,0.202963,0.202963,0.202963,0.202641,0.202641,10
